In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)

import itertools
import functools
import re


In [3]:
big_benchmarks_df = pd.read_pickle("benchmarks_by_budget.pkl")
big_means_df = big_benchmarks_df.groupby(by=["denoiser", "base_problem", "noise", "pop_size", "budget"]).mean().reset_index()

/var/folders/2c/tjd11sg11_51m4m2_swttz1h0000gn/T/ipykernel_12648/4196079501.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  big_means_df = big_benchmarks_df.groupby(by=["denoiser", "base_problem", "noise", "pop_size", "budget"]).mean().reset_index()


In [4]:
competitor_columns = ["Noisy", "R5", "R10", "R50", "R100", "AccumulativeResampling", "GPSS-mc2000", "GPSS-mc4000"]
knn_sort = [f"k{k}-MD{md}" for k, md in itertools.product([5, 10, 20, 50, 100], [0.1, 0.25, 0.5, 1])]
effect_sizes = {"negligible": 1, "small":2, "medium": 3, "large": 4}

def wilcoxon_pi_table(means_dataframe, pis, alpha_correct=192):
    approaches = means_dataframe.denoiser.unique()  # get a list of all approaches
    symbols_df = pd.DataFrame()  # the table we initialize
    for appA, appB in itertools.product(approaches, competitor_columns):  # iterate over pairs of approaches
        if appA in competitor_columns and appA != "baseline":
            continue
        if appA == appB:
            symbols_df.at[appA, appB] = ""
            continue
        # display(means_dataframe[means_dataframe.denoiser == appA])
        appA_df = means_dataframe[means_dataframe.denoiser == appA].sort_values(["budget", "base_problem", "noise", "pop_size"])
        appB_df = means_dataframe[means_dataframe.denoiser == appB].sort_values(["budget", "base_problem", "noise", "pop_size"])
        assert len(appA_df) == len(appB_df), f"both dataframes have the same length, currently it's {appA}: {len(appA_df)} vs {appB}: {len(appB_df)}"
        assert (appA_df["noise"].astype('string') + appA_df["pop_size"].astype('string')).tolist() == (appB_df["noise"].astype('string') + appB_df["pop_size"].astype('string')).tolist(), "sorting of dataframes is the same"
                
        pi_symbols = []
        for pi in pis:
            vd = None
            if ((appA_df[pi].reset_index() - appB_df[pi].reset_index()) == 0).all()[pi]:  # check if they are all zeros... 
                pi_symbols.append("=")
            elif wilcoxon(appA_df[pi], appB_df[pi])[1] >= (0.05 / alpha_correct):
                pi_symbols.append("=")
            else:
                vd = VD_A(list(appA_df[pi]), list(appB_df[pi]))
                if pi in ["perf_hv", "perf_ghv", "perf_rghv"]:
                    symbol = ("X" if vd[0] < 0.5 else "+") * effect_sizes[vd[1]]
#                     symbol = "x" if vd[0] < 0.5 else "+"
                else:
                    symbol = ("+" if vd[0] < 0.5 else "X") * effect_sizes[vd[1]]
#                     symbol = "+" if vd[0] < 0.5 else "x"

                pi_symbols.append(symbol)

        symbols_df.at[appA, appB] = "   ".join(pi_symbols)
    return (symbols_df.reindex(knn_sort))


# Create tables for total and split by feature (Noise, Pop Size, Budget, Problem)

Legend: 
- `++++` -> largely better
- `+++` -> moderately better
- `++` -> slightly better
- `+` -> negligibly better
- `=` -> equal
- `X` -> negligibly worse
- `XX` -> slightly worse
- `XXX` -> moderately worse
- `XXXX` -> largely worse


In [5]:
%%time
from scipy.stats import mannwhitneyu, wilcoxon
from VD_A import VD_A
import itertools

data = big_means_df
tables = []
print("Total")
for pi in ["IGD", "HV", "DeltaF"]:
    # print("total", pi)
    table = wilcoxon_pi_table(data, pis=[pi], alpha_correct=24*8)
    table.columns = pd.MultiIndex.from_product([[pi], table.columns])
    tables.append(table)
display(pd.concat(tables, axis=1))

slice_by_feature = {"noise": "Noise", "pop_size": "Pop Size", "budget": "Budget", "base_problem": "Problem"}
for column, feature in slice_by_feature.items():
    print("Slicing by feature", feature)
    for feature_value in big_means_df[column].unique():
        print(feature, feature_value)
        data = big_means_df[big_means_df[column] == feature_value]
        tables = []
        for pi in ["DeltaF", "IGD", "HV"]:
            table = wilcoxon_pi_table(data, pis=[pi], alpha_correct=24*8)
            table.columns = pd.MultiIndex.from_product([[pi], table.columns])
            tables.append(table)
        display(pd.concat(tables, axis=1))

Total


IGD                                                       \
            Noisy  R5 R10  R50 R100 AccumulativeResampling GPSS-mc2000   
k5-MD0.1        +  ++  ++  +++  +++                      +          ++   
k5-MD0.25       +  ++  ++  +++  +++                      +          ++   
k5-MD0.5        +  ++  ++  +++  +++                      +          ++   
k5-MD1          +  ++  ++  +++  +++                      +          ++   
k10-MD0.1       +  ++  ++  +++  +++                      +          ++   
k10-MD0.25      +  ++  ++  +++  +++                      +          ++   
k10-MD0.5       +  ++  ++  +++  +++                      +          ++   
k10-MD1         +  ++  ++  +++  +++                      +          ++   
k20-MD0.1       +  ++  ++  +++  +++                      +          ++   
k20-MD0.25      +  ++  ++  +++  +++                      +          ++   
k20-MD0.5       +  ++  ++  +++  +++                      +          ++   
k20-MD1         +  ++  ++  +++  +++                      +          ++   
k50-MD0.1       +  ++  ++  +++  +++                      +          ++   
k50-MD0.25      +  ++  ++  +++  +++                      +          ++   
k50-MD0.5       +  ++  ++  +++  +++                      +          ++   
k50-MD1         +  ++  ++  +++  +++                      +          ++   
k100-MD0.1      +  ++  ++  +++  +++                      +          ++   
k100-MD0.25     +  ++  ++  +++  +++                      +          ++   
k100-MD0.5      +  ++  ++  +++  +++                      +          ++   
k100-MD1        +  ++  ++  +++  +++                      +          ++   

                           HV                                         \
            GPSS-mc4000 Noisy R5 R10 R50 R100 AccumulativeResampling   
k5-MD0.1             ++     X  X   X  XX   XX                      X   
k5-MD0.25            ++     X  X   X  XX   XX                      X   
k5-MD0.5             ++     X  X   X  XX   XX                      X   
k5-MD1               ++     X  X   X  XX   XX                      X   
k10-MD0.1            ++     X  X   X  XX   XX                      X   
k10-MD0.25           ++     X  X   X  XX   XX                      X   
k10-MD0.5            ++     X  X   X  XX   XX                      X   
k10-MD1              ++     X  X   X  XX   XX                      X   
k20-MD0.1            ++     X  X   X  XX   XX                      X   
k20-MD0.25           ++     X  X   X  XX   XX                      X   
k20-MD0.5            ++     X  X   X  XX   XX                      X   
k20-MD1              ++     X  X   X  XX   XX                      X   
k50-MD0.1            ++     X  X   X  XX   XX                      X   
k50-MD0.25           ++     X  X   X  XX   XX                      X   
k50-MD0.5            ++     X  X   X  XX   XX                      X   
k50-MD1              ++     X  X   X  XX   XX                      X   
k100-MD0.1           ++     X  X   X  XX   XX                      X   
k100-MD0.25          ++     X  X   X  XX   XX                      X   
k100-MD0.5           ++     X  X   X  XX   XX                      X   
k100-MD1             ++     X  X   X  XX   XX                      X   

                                    DeltaF                          \
            GPSS-mc2000 GPSS-mc4000  Noisy    R5   R10   R50  R100   
k5-MD0.1              X           X     ++  XXXX  XXXX  XXXX  XXXX   
k5-MD0.25             X           X     ++  XXXX  XXXX  XXXX  XXXX   
k5-MD0.5              X           X    +++  XXXX  XXXX  XXXX  XXXX   
k5-MD1                X           X    +++  XXXX  XXXX  XXXX  XXXX   
k10-MD0.1             X           X     ++  XXXX  XXXX  XXXX  XXXX   
k10-MD0.25            X           X    +++  XXXX  XXXX  XXXX  XXXX   
k10-MD0.5             X           X    +++  XXXX  XXXX  XXXX  XXXX   
k10-MD1               X           X   ++++   XXX  XXXX  XXXX  XXXX   
k20-MD0.1             X           X     ++  XXXX  XXXX  XXXX  XXXX   
k20-MD0.25            X       

Slicing by feature Noise
Noise 0.1


DeltaF                                                             \
             Noisy    R5   R10   R50  R100 AccumulativeResampling GPSS-mc2000   
k5-MD0.1        ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.25      +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.5      ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD1        ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD1       ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD1       ++++   XXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD1       ++++   XXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.1      ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.25    +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.5     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD1       +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   

                          IGD                                            \
            GPSS-mc4000 Noisy  R5 R10  R50  R100 AccumulativeResampling   
k5-MD0.1           ++++     +  ++  ++  +++   +++                      +   
k5-MD0.25          ++++     +  ++  ++  +++  ++++                      +   
k5-MD0.5           ++++     +  ++  ++  +++  ++++                      +   
k5-MD1             ++++     +  ++  ++  +++  ++++                      +   
k10-MD0.1          ++++     +  ++  ++  +++   +++                      +   
k10-MD0.25         ++++     +  ++  ++  +++  ++++                      +   
k10-MD0.5          ++++     +  ++  ++  +++  ++++                      +   
k10-MD1            ++++     +  ++  ++  +++  ++++                      +   
k20-MD0.1          ++++     +  ++  ++  +++   +++                      +   
k20-MD0.25         ++++     +  ++  ++  +++  ++++                      +   
k20-MD0.5          ++++     +  ++  ++  +++  ++++                      +   
k20-MD1            ++++     +  ++  ++  +++  ++++                      +   
k50-MD0.1          ++++     +  ++  ++  +++   +++                      +   
k50-MD0.25         ++++     +  ++  ++  +++  ++++                      +   
k50-MD0.5          ++++     +  ++  ++  +++  ++++                      +   
k50-MD1            ++++     =  ++  ++  +++  ++++                      +   
k100-MD0.1         ++++     +  ++  ++  +++   +++                      +   
k100-MD0.25        ++++     +  ++  ++  +++  ++++                      +   
k100-MD0.5         ++++     +  ++  ++  +++  ++++                      +   
k100-MD1           ++++     =  ++  ++  +++  ++++                      +   

                                       HV                  \
            GPSS-mc2000 GPSS-mc4000 Noisy R5 R10 R50 R100   
k5-MD0.1             ++          ++     X  X   X  XX   XX   
k5-MD0.25            ++          ++     X  X   X  XX   XX   
k5-MD0.5             ++          ++     X  X   X  XX   XX   
k5-MD1               ++          ++     X  X   X  XX   XX   
k10-MD0.1            ++          ++     X  X   X  XX   XX   
k10-MD0.25           ++          ++     X  X   X  XX   XX   
k10-MD0.5            ++          ++     X  X   X  XX   XX   
k10-MD1              ++      

Noise 0.25


...

Problem truss2d


DeltaF                                                             \
             Noisy    R5   R10   R50  R100 AccumulativeResampling GPSS-mc2000   
k5-MD0.1      XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.25     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.5      XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD1        XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.1     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.25    XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.5     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD1       XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.1     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.25    XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.5     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD1       XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.1     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.25    XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.5     XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD1       XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.1    XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.25   XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.5    XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD1      XXXX  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   

                          IGD                                                 \
            GPSS-mc4000 Noisy    R5   R10   R50  R100 AccumulativeResampling   
k5-MD0.1           ++++  ++++   XXX  XXXX  XXXX  XXXX                   ++++   
k5-MD0.25          ++++  ++++     =  XXXX  XXXX  XXXX                   ++++   
k5-MD0.5           ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k5-MD1             ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k10-MD0.1          ++++  ++++   XXX  XXXX  XXXX  XXXX                   ++++   
k10-MD0.25         ++++  ++++     =  XXXX  XXXX  XXXX                   ++++   
k10-MD0.5          ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k10-MD1            ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k20-MD0.1          ++++  ++++   XXX  XXXX  XXXX  XXXX                   ++++   
k20-MD0.25         ++++  ++++     =  XXXX  XXXX  XXXX                   ++++   
k20-MD0.5          ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k20-MD1            ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k50-MD0.1          ++++  ++++   XXX  XXXX  XXXX  XXXX                   ++++   
k50-MD0.25         ++++  ++++     =  XXXX  XXXX  XXXX                   ++++   
k50-MD0.5          ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k50-MD1            ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k100-MD0.1         ++++  ++++   XXX  XXXX  XXXX  XXXX                   ++++   
k100-MD0.25        ++++  ++++     =  XXXX  XXXX  XXXX                   ++++   
k100-MD0.5         ++++  ++++  ++++     =  XXXX  XXXX                   ++++   
k100-MD1           ++++  ++++  ++++     =  XXXX  XXXX                   ++++   

                                       HV                          \
            GPSS-mc2000 GPSS-mc4000 Noisy    R5   R10   R50  R100   
k5-MD0.1           XXXX        XXXX  XXXX     =  ++++  ++++  ++++   
k5-MD0.25          XXXX        XXXX  XXXX     =   +++   +++   +++   
k5-MD0.5              =           =  XXXX  XXXX    XX     =     =   
k5-MD1                =           =  XXXX  XXXX     =     =     =   
k10-MD0.1          XXXX        XXXX  XXXX     =  ++++ 

Problem welded_beam


DeltaF                                                             \
             Noisy    R5   R10   R50  R100 AccumulativeResampling GPSS-mc2000   
k5-MD0.1        ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.25     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD0.5      ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k5-MD1        ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k10-MD1       ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k20-MD1       ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.1       ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.25     +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD0.5     ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k50-MD1       ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.1      ++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.25    +++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD0.5    ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   
k100-MD1      ++++  XXXX  XXXX  XXXX  XXXX                   XXXX        ++++   

                          IGD                                                 \
            GPSS-mc4000 Noisy    R5   R10   R50  R100 AccumulativeResampling   
k5-MD0.1           ++++  ++++     =     =  ++++  ++++                   ++++   
k5-MD0.25          ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k5-MD0.5           ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k5-MD1             ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k10-MD0.1          ++++  ++++     =     =  ++++  ++++                   ++++   
k10-MD0.25         ++++  ++++   +++   +++  ++++  ++++                   ++++   
k10-MD0.5          ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k10-MD1            ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k20-MD0.1          ++++  ++++     =     =  ++++  ++++                   ++++   
k20-MD0.25         ++++  ++++   +++   +++  ++++  ++++                   ++++   
k20-MD0.5          ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k20-MD1            ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k50-MD0.1          ++++  ++++     =     =  ++++  ++++                   ++++   
k50-MD0.25         ++++  ++++   +++  ++++  ++++  ++++                   ++++   
k50-MD0.5          ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k50-MD1            ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k100-MD0.1         ++++  ++++     =     =  ++++  ++++                   ++++   
k100-MD0.25        ++++  ++++   +++  ++++  ++++  ++++                   ++++   
k100-MD0.5         ++++  ++++  ++++  ++++  ++++  ++++                   ++++   
k100-MD1           ++++  ++++  ++++  ++++  ++++  ++++                   ++++   

                                       HV                          \
            GPSS-mc2000 GPSS-mc4000 Noisy    R5   R10   R50  R100   
k5-MD0.1              =           =   XXX  XXXX  XXXX  XXXX  XXXX   
k5-MD0.25          ++++        ++++  XXXX  XXXX  XXXX  XXXX  XXXX   
k5-MD0.5           ++++        ++++  XXXX  XXXX  XXXX  XXXX  XXXX   
k5-MD1             ++++        ++++  XXXX  XXXX  XXXX  XXXX  XXXX   
k10-MD0.1             =           =   XXX  XXXX  XXXX 

CPU times: user 1min 17s, sys: 1.06 s, total: 1min 18s
Wall time: 1min 18s
